# K Nearest Neighbors Graphs

[Nearest neighbor graphs](https://en.wikipedia.org/wiki/Nearest_neighbor_graph) are fundamental to work around outlier or anomaly detection, as well as implementing more sophisticated graph algorithms. Although Neo4J includes similarity metrics out of the box, constructing the nearest neighbors graph would require a brute force calculation, calculating pairwise similarity for *every* pair of nodes in the graph and then constructing a new graph based on those results. This is fine for small graphs, but prohibitively slow on real world data sets. 

To prototype some examples of how this can be done with a combination of Neo4J and Python, I'm looking at examples from four popular nearest neighbor libraries: sklearn's neighbors, and three approximate NN libraries (faster run times on large data) NMSLib, annoy, and pynndescent. Examples are below.

## SKLearn / NearestNeighbors
SKLearn supports NearestNeighbors implements unsupervised nearest neighbors learning. It acts as a uniform interface to three different nearest neighbors algorithms: BallTree, KDTree, and a brute-force algorithm based on routines in sklearn.metrics.pairwise - it does not allow specifying the distance function. In contrast to the subsequent libraries, this is an exact calculation so it may not scale well.

### KD Tree: 
The KD tree is a binary tree structure which recursively partitions the parameter space along the data axes, dividing it into nested orthotropic regions into which data points are filed.

### BallTree:
Where KD trees partition data along Cartesian axes, ball trees partition data in a series of nesting hyper-spheres. This makes tree construction more costly than that of the KD tree, but results in a data structure which can be very efficient on highly structured data, even in very high dimensions.

### example from the documentation
**input:** numpy array

**output:** list object with two arrays per input vector: one containing the IDs of the K-nearest neighbor nodes, and one containing the distances to those neighbors.

In [27]:
import numpy as np
from sklearn.neighbors import KDTree, BallTree
from sklearn.preprocessing import normalize

In [25]:
nn_data = np.random.uniform(0, 1, size=(1000, 5))
tree = KDTree(nn_data)
tree_indices = tree.query(nn_data, 10, return_distance=False)

In [28]:
balltree = BallTree(nn_data)
tree_indices = tree.query(nn_data, 10, return_distance=False)

## nmslib
Non-Metric Space Library (NMSLIB): An efficient similarity search library and a toolkit for evaluation of k-NN methods for generic non-metric spaces. This is a python wrapper for a C++ library. Methods in nmslib (specifically hnsw) are consistently the best performing algorithms in the [ANN benchmark dataset](http://ann-benchmarks.com/)

### example from the documentation
**input:** numpy array

**output:** list object with two arrays per input vector: one containing the IDs of the K-nearest neighbor nodes, and one containing the distances to those neighbors.

In [10]:
import nmslib

In [9]:
# create a random matrix to index
data = numpy.random.randn(10000, 100).astype(numpy.float32)

# initialize a new index, using a HNSW index on Cosine Similarity
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)

# query for the nearest neighbours of the first datapoint
ids, distances = index.knnQuery(data[0], k=10)

# get all nearest neighbours for all the datapoint using a pool of 4 threads to compute
# outputs a list object with one row listing the ids of the n-nearest neighbors and the next row listing the distances
neighbours = index.knnQueryBatch(data, k=10, num_threads=4)


In [ ]:
# some code

## PyNNDescent

PyNNDescent is a python library that implements the algorithm described in [Efficient K-Nearest Neighbor Graph Construction for Generic Similarity Measures](http://www.cs.princeton.edu/cass/papers/www11.pdf). It uses random projection trees for initialisation with a variety of metrics that are amenable to such approaches (euclidean, minkowski, angular, cosine, etc.). While it is not the top performing library in the [ANN Benchmark](http://ann-benchmarks.com/) it is written in python and could be easily forked to make use of Neo4J's internal similarity calculations, obviating the need to use an embedding and export from the graph.

### Example from the documentation
This codebase is a bit sparse and doesn't really have examples, so I'm guessing at the correct usage here... *fingers crossed* that this is reasonable.

**input:** numpy array of vectors

**output:** two numpy arrays, one with the indices (per row) and one with the distance

In [22]:
from pynndescent import NNDescent

In [14]:
nn_data = np.random.uniform(0, 1, size=(1000, 5))

In [15]:
knn_indices, knn_dists = NNDescent(
        nn_data, "euclidean", {}, 10, random_state=np.random
    )._neighbor_graph

In [21]:
knn_indices.shape

(1000, 10)

## Annoy
Annoy (Approximate Nearest Neighbors Oh Yeah) is a C++ library with Python bindings to search for points in space that are close to a given query point. It also creates large read-only file-based data structures that are mmapped into memory so that many processes may share the same data. Annoy is produced by spotify, to power their recommendation engines. Annoy is almost as fast as the fastest libraries, but there is actually another feature that really sets Annoy apart: it has the ability to use static files as indexes. In particular, this means you can share index across processes. Annoy also decouples creating indexes from loading them, so you can pass around indexes as files and map them into memory quickly.

### Example from the documentation
**input**: vectors that describe each object which are used to populate the index

**output:** ANN model, which can then be passed an index and return the k nearest neighbors

In [31]:
from annoy import AnnoyIndex
import random

# create an annoy index which has one item vector for each item to be used in the calculations. 
f = 40
t = AnnoyIndex(f)  # Length of item vector that will be indexed
for i in range(1000): # this builds an index with 1000 items which are each 40 digits long
    v = [random.gauss(0, 1) for z in range(f)]
    t.add_item(i, v)


In [39]:
# next you build an ANN model - you can specify the number of trees - more trees = better model, but slower
t.build(10) # 10 trees
t.save('test.ann')

True

In [41]:
# now to calculate neighbors, you load the model into an index and then use the get nns by item function.
u = AnnoyIndex(f)
u.load('test.ann') # super fast, will just mmap the file
print(u.get_nns_by_item(1, 10)) # will find the 10 nearest neighbors

[1, 919, 139, 814, 632, 143, 941, 388, 413, 308]


## how can I use any of this with Neo4J?

The simplest integration is to calculate some sort of node embedding, use those vectors for distance calculations, and then push the resulting data into Neo4J as a new graph. This is somewhat complicated (Neo4J doesn't really support it's embeddings, it's not technically possible to have to databases up at the same time) but let's try to make this work...

A fancier solution might be to fork one of these libraries (PyNNDescent looks the easiest) and replace the internal distance calculations with calls to the in memory graph.

Let's start by running the DeepGL embeddings on the GoT graph. Note that to use these embeddings, we'll need to download the jar file from https://github.com/neo4j-graph-analytics/ml-models/releases and adding the JAR file into the plugins folder of the database. Add the contents of package embeddings to the recognized procedures at the bottom of Manage->Settings. If you already have APOC and Graph Algorithms installed, it will look something like this: `dbms.security.procedures.whitelist=algo.*,apoc.*,embeddings.*`

In [ ]:
# let's try running the DeepGL embedding on the GoT graph
# note that to run this, you'll need to download the ml-models jar from